In [1]:
import gzip
import os
from sklearn.ensemble import RandomForestRegressor
import numpy as np

from collections import defaultdict
def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

items = []
users = []
ratings = []
nHelpFuls = []
outOfs = []
texts = []
times = []
counter = 0
testindex = 0

for l in readGz("train.json.gz"):
    items.append(l['itemID'])
    users.append(l['reviewerID'])
    ratings.append(l['rating'])
    nHelpFuls.append(l['helpful']['nHelpful'])
    outOfs.append(l['helpful']['outOf'])
    texts.append(l['reviewText'])
    times.append(l['unixReviewTime'])
    counter += 1
    if counter == 900000:
        testindex = len(ratings)

os.system('say "Data Loaded"')

0

In [2]:
itemsdict = {}
itemsavg = {}
usersdict = {}
usersavghelpful = {}
userrevcount = defaultdict(int)

for i in xrange(len(ratings)):
    if items[i] not in itemsdict:
        itemsdict[items[i]] = []
    if times[i] not in itemsdict[items[i]]:
        itemsdict[items[i]].append(times[i])

for i in xrange(len(ratings)):
    if items[i] not in itemsavg:
        itemsavg[items[i]] = []
    itemsavg[items[i]].append(ratings[i])
    
for i in xrange(len(ratings)):
    if users[i] not in usersdict:
        usersdict[users[i]] = []
    if times[i] not in usersdict[users[i]]:
        usersdict[users[i]].append(times[i])
        
for i in xrange(len(ratings)):
    if outOfs[i] != 0:
        if users[i] not in usersavghelpful:
            usersavghelpful[users[i]] = []
        usersavghelpful[users[i]].append(nHelpFuls[i]/outOfs[i])

for i in xrange(len(ratings)):
    userrevcount[users[i]] += 1
    
#kaggle data loading...
kitems = []
kusers = []
kratings = []
koutOfs = []
ktexts = []
ktimes = []
for l in readGz("helpful.json.gz"):
    kitems.append(l['itemID'])
    kusers.append(l['reviewerID'])
    kratings.append(l['rating'])
    koutOfs.append(l['helpful']['outOf'])
    ktexts.append(l['reviewText'])
    ktimes.append(l['unixReviewTime'])

for i in xrange(len(kratings)):
    if kitems[i] not in itemsdict:
        itemsdict[kitems[i]] = []
    if ktimes[i] not in itemsdict[kitems[i]]:
        itemsdict[kitems[i]].append(ktimes[i])

for i in xrange(len(kratings)):
    if kitems[i] not in itemsavg:
        itemsavg[kitems[i]] = []
    itemsavg[kitems[i]].append(kratings[i])

for key in itemsdict:
    itemsdict[key].sort()
os.system('say "Kaggle data loaded"')

for key in itemsavg:
    itemsavg[key] = sum(itemsavg[key])/len(itemsavg[key])
    
for i in xrange(len(kratings)):
    if kusers[i] not in usersdict:
        usersdict[kusers[i]] = []
    if ktimes[i] not in usersdict[kusers[i]]:
        usersdict[kusers[i]].append(ktimes[i])

for key in usersdict:
    usersdict[key].sort()

for key in usersavghelpful:
    usersavghelpful[key] = sum(usersavghelpful[key])/len(usersavghelpful[key])

In [21]:
train = []
outoftrain = []
target = []
test = []
outoftest = []
targettest = []
for i in xrange(len(ratings)):
    if outOfs[i] != 0:
        row = []
        #ratings
        row.append(ratings[i])
        
        #wc
        wc = texts[i].split()
        row.append(len(wc))
        
        row.append(times[i])
        row.append(outOfs[i])
        
        #MoreFeatures
        #sentences
        sc = texts[i].split('. ')
        #row.append(len(sc))
        
        #avg words
        b = float(len(wc))/len(sc)
        row.append(b)
        #avg chars
        c = [len(w) for w in wc]
        c = 0 if len(c) == 0 else float(sum(c))/len(c)
        #row.append(c)
        #readability
        d = 4.71*c + 0.5*b - 21.43
        row.append(d)
        
        #timed rank
        # scale and try -val to + val etc... try scale next
        rank = (len(itemsdict[items[i]])+1)/2.0 - (itemsdict[items[i]].index(times[i])+1)
        row.append(rank)
        
        #time difference
        diff = times[i] - itemsdict[items[i]][0]
        row.append(diff)
        
        #deviation
        dev = abs(ratings[i] - itemsavg[items[i]])
        row.append(dev)
        row.append(dev**2)
        
        #user properties
        #timed rank
        #rank = (len(usersdict[users[i]])+1)/2.0 - (usersdict[users[i]].index(times[i])+1)
        #row.append(rank)
        
        #avg user helpfulness
        #row.append(usersavghelpful[users[i]])
        
        #user rev count
        #row.append(userrevcount[users[i]])
        
        #user product helpfulness count
        row.append(userrevcount[users[i]]*usersavghelpful[users[i]])
        
        #if i < testindex:
        train.append(row)
        target.append(nHelpFuls[i]/float(outOfs[i]))
        outoftrain.append(outOfs[i])
        #else:
        #    test.append(row)
        #    targettest.append(nHelpFuls[i]/float(outOfs[i]))
        #    outoftest.append(outOfs[i])
            
rf = RandomForestRegressor(n_estimators=200, n_jobs=-1)
rf.fit(train, target)
os.system('say "Data Trained"')

0

In [ ]:
trainpredict = rf.predict(train)
testpredict = rf.predict(test)

errors = []
for i in range(len(trainpredict)):
    errors.append(abs(target[i] - trainpredict[i])*outoftrain[i])
mae = float(sum(errors))/len(errors)
print mae

errors = []
for i in range(len(testpredict)):
    errors.append(abs(targettest[i] - testpredict[i])*outoftest[i])
mae = float(sum(errors))/len(errors)
print mae
os.system('say "Errors Updated"')

"""
mae corrected
0.453925027129
1.15620196629
"""
"""
0.45537700253
1.15210030907
"""
"""
0.449873481314
1.14146121432
"""
"""
len-rank for rank
0.454805464773
1.12437800129
"""
"""
len/rank as rank, default estimators
0.463616939687
1.12498733316
"""
"""
len - rank, 100 estimators
0.415570064647
1.077622777
"""
"""
0.398592495018: no c, len-rank = rank
1.05560638633
"""
"""
try with nhelpfuls instead of ratio
0.41684491092
1.08506242674
"""
"""
ratio, rank rearranged
0.399153271742
1.05442916258
"""
"""
ratio, rank scaled:bekar
"""
"""
time diff
0.396901957506
1.04032779202
"""
"""
deviation
0.374313124583
0.989569597072
"""
"""
all 1000000 rows
"""

In [22]:
userItemPrediction = defaultdict(list)
kaggletest = []

for i in range(len(kratings)):
    row = []
    #ratings
    row.append(kratings[i])

    #wc
    wc = ktexts[i].split()
    row.append(len(wc))

    row.append(ktimes[i])
    row.append(koutOfs[i])

    #MoreFeatures
    #sentences
    sc = ktexts[i].split('. ')
    #row.append(len(sc))

    #avg words
    b = float(len(wc))/len(sc)
    row.append(b)
    #avg chars
    c = [len(w) for w in wc]
    c = 0 if len(c) == 0 else float(sum(c))/len(c)
    #row.append(c)
    #readability
    d = 4.71*c + 0.5*b - 21.43
    row.append(d)

    #timed rank
    rank = (len(itemsdict[kitems[i]])+1)/2.0 - (itemsdict[kitems[i]].index(ktimes[i])+1)
    row.append(rank)
    
    #time difference
    diff = ktimes[i] - itemsdict[kitems[i]][0]
    row.append(diff)

    #deviation
    dev = abs(kratings[i] - itemsavg[kitems[i]])
    row.append(dev)
    row.append(dev**2)
    
    #user properties
    #timed rank
    #rank = (len(usersdict[kusers[i]])+1)/2.0 - (usersdict[kusers[i]].index(ktimes[i])+1)
    #row.append(rank)

    #avg user helpfulness
    if kusers[i] in usersavghelpful:
        row.append(usersavghelpful[kusers[i]])
    else:
        row.append(0.0)
    
    #user rev count
    #row.append(userrevcount[kusers[i]])
    
    #user product helpfulness count
    if kusers[i] in usersavghelpful:
        row.append(userrevcount[kusers[i]]*usersavghelpful[kusers[i]])
    else:
        row.append(0.0)
    
    kaggletest.append(row)

kagglepredict = rf.predict(kaggletest)
for i in range(len(kusers)):
    userItemPrediction[kusers[i]+'-'+kitems[i]+'-'+str(koutOfs[i])] = koutOfs[i]*kagglepredict[i]

f_out = open('outputTry1.txt', 'w')
f = open('pairs_Helpful.txt', 'r')
f_out.write(f.readline())
for l in f.readlines():
    f_out.write(l.rstrip('\n')+','+str(userItemPrediction[l.rstrip('\n')])+'\n')
f_out.close()
f.close()
os.system('say "Kaggle calculated"')

0

In [13]:
a = 8.63333

In [16]:
round(a)

9.0

In [ ]:
#Training the model: Linear
from scipy.optimize import minimize
def mse (a, helpfulness, f1, f2, f3):
    #helpfulness = nHelpful/outOf
    squared_error = 0
    for i in range(len(helpfulness)):
        squared_error += (helpfulness[i] - a[0] - a[1]*f1[i] - a[2]*f2[i])**2
    mse = squared_error/len(helpfulness)
    return mse

f1 = train[:,1]
f2 = train[:,2]
f3 = train[:,3]
res = minimize(mse, [0]*(len(train[0])-1), args=(target, f1, f2, f3), method='L-BFGS-B')
print res

a = res.x
errors = []
for i in range(len(alpha_array_test)):
    errors.append(abs(nHelpful_test[i] - (a[0] + a[1]*wordCount_test[i] + a[2]*ratings_test[i])*outOf_test[i]))
mae = sum(errors)/len(errors)
print mae

In [ ]:
#<Q4>
import gzip
from collections import defaultdict

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

### Rating baseline: compute averages for each user, or return the global average if we've never seen the user before

userItemPrediction = defaultdict(list)

userIds = []
itemIds = []
outOfs = []
wordCounts = []
ratings = []

for l in readGz("helpful.json.gz"):
    user,item,outOf,rating,wordCount = l['reviewerID'],l['itemID'],l['helpful']['outOf'],l['rating'],len(l['reviewText'].split())
    outOfs.append(outOf)
    userIds.append(user)
    itemIds.append(item)
    wordCounts.append(wordCount)
    ratings.append(rating)

a = res.x
userItemPrediction = defaultdict(list)

for i in range(len(userIds)):
    userItemPrediction[userIds[i]+'-'+itemIds[i]+'-'+str(outOfs[i])] = outOfs[i]*(a[0]+a[1]*wordCounts[i]+a[2]*ratings[i])

f_out = open('output.txt', 'w')
f = open('pairs_Helpful.txt', 'r')
f_out.write(f.readline())
for l in f.readlines():
    f_out.write(l.rstrip('\n')+','+str(userItemPrediction[l.rstrip('\n')])+'\n')
f_out.close()
f.close()

#Kaggle user name/Id: apoorvedave/458740 
#</Q4>

In [ ]:
#<Q5>
rating_alpha = sum(ratings_train)/len(ratings_train)
#4.0894368581424221

In [ ]:
#<Q6>
file_read_counter = 0
users_train = []
items_train = []
ratings_train = []

users_test = []
items_test = []
ratings_test = []

user_item_ratings_train = {}
item_user_ratings_train = {}
user_item_ratings_test = {}
count = 0
for l in readGz("train.json.gz"):
    if file_read_counter < 100000:
        users_train.append(l['reviewerID'])
        items_train.append(l['itemID'])
        ratings_train.append(l['rating'])
    elif file_read_counter >= 900000:
        users_test.append(l['reviewerID'])
        items_test.append(l['itemID'])
        ratings_test.append(l['rating'])
            
    file_read_counter += 1
    if file_read_counter %10000 == 0:
        print file_read_counter

In [7]:
for i in range(len(users_train)):
    if users_train[i] not in user_item_ratings_train:
        user_item_ratings_train[users_train[i]] = {}
    user_item_ratings_train[users_train[i]][items_train[i]] = ratings_train[i]
    if items_train[i] not in item_user_ratings_train:
        item_user_ratings_train[items_train[i]] = {}
    item_user_ratings_train[items_train[i]][users_train[i]] = ratings_train[i]

In [88]:
alpha = 1
set_users = set(users_train)
set_items = set(items_train)

list_users = list(set_users)
list_items = list(set_items)
lam = 1

beta_u = defaultdict(float)
beta_i = defaultdict(float)

for i in range(len(set_users)):
    beta_u[list_users[i]] = 0
for i in range(len(set_items)):
    beta_i[list_items[i]] = 0

old_alpha = 10

for i in range(len(set_users)):
    beta_u[list_users[i]] = 0
for i in range(len(set_items)):
    beta_i[list_items[i]] = 0
old_alpha = 10

while abs(alpha - old_alpha) > 0.001:
    old_alpha = alpha
    alpha = 0
    for i in range(len(ratings_train)):
        alpha += ratings_train[i] - beta_u[users_train[i]] - beta_i[items_train[i]]
    alpha = alpha/len(ratings_train)

    for user in beta_u:
        beta_u[user] = 0
        for item in user_item_ratings_train[user]:
            beta_u[user] += user_item_ratings_train[user][item] - alpha - beta_i[item]
        beta_u[user] = beta_u[user]/(lam+len(user_item_ratings_train[user]))

    for item in beta_i:
        beta_i[item] = 0
        for user in item_user_ratings_train[item]:
            beta_i[item] += item_user_ratings_train[item][user] - alpha - beta_u[user]
        beta_i[item] = beta_i[item]/(lam + len(item_user_ratings_train[item]))

In [71]:
def mse_converge(lam):
    global alpha, beta_u, beta_i,users_train,items_train,ratings_train, user_item_ratings_train 
    alpha = 0
    for i in range(len(set_users)):
        beta_u[list_users[i]] = 0
    for i in range(len(set_items)):
        beta_i[list_items[i]] = 0
    old_alpha = 10
    
    while abs(alpha - old_alpha) > 0.001:
        old_alpha = alpha
        alpha = 0
        for i in range(len(ratings_train)):
            alpha += ratings_train[i] - beta_u[users_train[i]] - beta_i[items_train[i]]
        alpha = alpha/len(ratings_train)

        for user in beta_u:
            beta_u[user] = 0
            for item in user_item_ratings_train[user]:
                beta_u[user] += user_item_ratings_train[user][item] - alpha - beta_i[item]
            beta_u[user] = beta_u[user]/(lam+len(user_item_ratings_train[user]))

        for item in beta_i:
            beta_i[item] = 0
            for user in item_user_ratings_train[item]:
                beta_i[item] += item_user_ratings_train[item][user] - alpha - beta_u[user]
            beta_i[item] = beta_i[item]/(lam + len(item_user_ratings_train[item]))

In [87]:
m=-400
u=''
for k in beta_i:
    if m < beta_i[k]:
        m = beta_i[k]
        u = k
m,u

(1.2495251818321746, 'I635442408')

In [28]:
errors = []
for i in range(len(ratings_train)):
    errors.append(abs(ratings_train[i] - alpha - beta_u[users_train[i]] - beta_i[items_train[i]])**2)
sum(errors)/len(errors)

errors = []
for i in range(len(ratings_test)):
    errors.append(abs(ratings_test[i] - alpha - beta_u[users_test[i]] - beta_i[items_test[i]])**2)
sum(errors)/len(errors)
#MSE for training Set = 0.3335585596679398
#MSE for test Set = 0.9042631835049043
#alpha 4.2003778800008265

In [32]:
sum(errors)/len(errors)

0.9042631835049043

In [75]:
errors = []
for i in range(len(ratings_test)):
    errors.append(abs(ratings_test[i] - alpha - beta_u[users_test[i]] - beta_i[items_test[i]])**2)
print sum(errors)/len(errors)

0.838915319212


In [63]:
alpha

4.21824

In [68]:
import scipy.optimize
lam = 1
def find_lam(lam):
    mse_converge(lam)
    return valid_mse()
x = scipy.optimize.minimize(find_lam,[1],args=(),method="BFGS")



[1]


KeyError: 'U924592645'

In [76]:

def model_predict(i,u):
    return (alpha + beta_u[u] + beta_i[i] )

predictions = open("predictions_Ratings.txt", 'w')

for l in open("pairs_Rating.txt"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    pred = model_predict(i,u)
    predictions.write(u + '-' + i + ',' + str(pred) + '\n')

predictions.close()

#0.838915319212

In [89]:
alpha

4.2003778800008265

In [90]:
errors = []
for i in range(len(ratings_test)):
    errors.append((ratings_test[i] - alpha - beta_u[users_test[i]] - beta_i[items_test[i]])**2)


In [91]:
sum(errors)/len(errors)

0.9042631835049043

In [20]:
a = "THis is s acengesl. aldkf.".split('.')
a

['THis is s acengesl', ' aldkf', '']

In [48]:
import os

In [49]:
os.system('say "Hello"')

0

In [63]:
a = 'This book "succubi" me in from the first paragraph. Jackie Brigton wakes up in a dumpster to find some where she lost a whole day. What more she on the opposite side of town from where she lives and with an actual Angel interested in her. Seems the combination of a night spent with both an angel and a vampire has turned her into a sex demon known as a succubi. Angel Noah introduce Jackie to Remy a porn star Succubi who happy to have one of her own in town. Unfortunately her world has only started to turn upside down as she now has two masters who have complete over her (an angel and a vampire.)  Jackie and her new immortal friends find themselves in the middle of an angel/vampire feud of power. Both sides wanting a halo which if either side hand would end the world as Jackie knows it. In order so save Noah\'s life Jackie find herself in a no win situation with the world itself at stake.'
a = [len(i.split()) for i in a.split('. ')]
a = sum(a)/len(a)

In [66]:
a

21